# python模块的导入

编写单个脚本文件时，只需进行简单的常规导入即可满足需求；在编写多个python文件时，通常需要进行自定义模块调用，这就会出现比较复杂的调用需求，比如同包不同文件以及不同包不同文件的调用等等。下面我们就来详细探讨一些python包的导入方式。参考链接：<http://codingpy.com/article/python-import-101/>.

### 常规导入(Regular imports)
这是最常用、最简单的导入方式。这里先给出一些常规导入的例子：
```python
# (1)
import sys
# (2)
import os, sys, time
# (3)
import sys as system
print system.platform
# (4)
import urllib.error
```
***方式(1):***用于导入系统自带的模块、已经安装到系统中的第三方模块以及当前目录下的自定义模块。
***方式(2):***在一条语句中导入多个模块。
***方式(3):*** import A as B, 用于导入模块A，并将该模块命名为B。 
***方式(4):*** import A.b, 用于导入A模块中的b子模块。

### from导入(From imports)
有些情况下， 我们想要从某个模块中导入一部分子模块，这时from\dots import\dots 的方式就能够满足这样的需求。下面我们先来看看用该方式导入的例子。
```python
# (1)
from os import path
# (2)
from os import *
# (3)
from os import path, walk, unlink
from os import uname, remove
# (4)
from os import (path, walk, ulink, 
                uname, remove)
# (5)
from os import path, walk, ulink,\
               uname, remove
```
***方式(1):***用于从模块中导入一个子模块，{\bf{方式(2):}}用于从一个模块中导入所有子模块，但该导入方式会破坏命名空间，增大模块重名冲突的几率。{\bf{方式(3-5):}}都是从模块中导入若干个子模块。

### 相对导入(Relative imports)
使用句点的方式来相对导入其他的包或模块可以避免偶然情况下导入标准库中的同名模块而造成的冲突。基于PEP 328给出的文件夹结构，几个相对导入的例子如下：
```python
"""
The structure of folders:
my_package/
	__init__.py
	subpacjage1/
		__init__.py
		module_x.py
		module_y.py
	subpackage2/
		__init__.py
		module_z.py
	module_a.py
"""
# (1) Write in my_package/__init__.py
from . import subpackage1
from . import subpackage2

# (2) Write in subpackage1/__init__.py
from . import module_x
from . import module_y

# (3) Write in module_x.py
from .module_y import spam as ham

def main():
	ham()

# (4) Write in module_y.py
def spam():
	print('spam' * 3)
```
以上代码编写完成以后，就可以进行导入运行，步骤如下：\\
(1) 在终端或命令行中进入到my_package的父目录，然后进入Python交互环境\\
(2) 运行 import my\_package\\
(3) 运行 my_package.subpackage1.module_x.main()\\
(4) 会输出：spam spam spam

到此，相对导入的运行流程和基本方式已经明白，下面我们还需要对几种报错的情况进行解析。\\
(1) 必须在my_package文件夹的父目录下进行导入，在别的目录下，将会找不到该Python包。所谓Python包是指，一个包含__init__.py文件的文件夹。该文件可以写入一些模块导入和参数初始化工作的代码,也可以为空，但必须存在。\\
(2) module_x.py文件只能以模块的方式运行，不能以py文件的方式运行，否则报错，因为脚本模式下是不支持相对导入的。我们可以在module_x.py加入一段代码
```python
if __name__ == '__main__':
	main()
```
然后直接运行该文件，来重现报错的现象。\\
(3) 如果需要调用该模块，但相对路径无法使用的情况下，需要将该包的父目录加到Python的搜索目录下，采用代码如下：
```python
import sys
sys.path.append('/path/to/folder/containing/my_package')
import my_package
```

### 可选导入(Optional imports)
如果想在默认情况下使用一些包，但是在这些包缺失的情况下又能使用备用的包，那么可选导入就能派上用场，github2包中可选导入的例子如下：
```python
try:
# For Python 3
    from http.client import responses
except ImportError:  # For Python 2.5-2.7
    try:
        from httplib import responses  # NOQA
    except ImportError:  # For Python 2.4
        from BaseHTTPServer import BaseHTTPRequestHandler as _BHRH
        responses = dict([(k, v[0]) for k, v in _BHRH.responses.items()])
```

### 局部导入(Local imports)
先来看一个全局导入和局部导入的例子：
```python
import sys  # global scope

def square_root(a):
	# This import is into the square_root functions local scope
	import math
	return math.sqrt(a)

def my_pow(base_num, power):
	return math.pow(base_num, power)

if __name__ == '__main__':
	print(square_root(49))
	print(my_pow(2, 3))
```
其中sys模块就是全局导入，因此该文件内的任何函数都能使用该模块；而math模块就是局部导入，只能在square_root函数中使用。如果我们试图调用my_pow函数，将会报错，因为该函数调用未被导入的模块math。需要说明的是，除非必要，尽量避免使用局部导入，因为这会让代码的可读性和可维护性大打折扣。一般我们会遵守标准，将所有的导入放在文件的开头部分。

另外一些可能出现的导入错误还有循环导入和覆盖导入。循环导入是指A导入B，B也需要导入A，这将会导致报错。对于这种情况，首先考虑修改代码，除非必要，切勿使用。覆盖导入是指自定义模块与系统木块重名，而Python是优先搜索当前目录，这就会导致自定义木块覆盖系统模块，可能导致调用报错。